# Geometry optimization

We use DFTK and the [GeometryOptimization](https://github.com/JuliaMolSim/GeometryOptimization.jl/)
package to find the minimal-energy bond length of the $H_2$ molecule.
First we set up an appropriate `DFTKCalculator` (see AtomsCalculators integration),
for which we use the LDA model just like in the Tutorial for silicon
in combination with a pseudodojo pseudopotential (see Pseudopotentials).

In [1]:
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbe.v0_4_1.oncvpsp3.standard.upf")
calc = DFTKCalculator(;
    model_kwargs = (; functionals=LDA(), pseudopotentials),  # model_DFT keyword arguments
    basis_kwargs = (; kgrid=[1, 1, 1], Ecut=10)  # PlaneWaveBasis keyword arguments
)

DFTKCalculator(functionals=Xc(lda_x, lda_c_pw), pseudopotentials=PseudoFamily("dojo.nc.sr.pbe.v0_4_1.oncvpsp3.standard.upf"), Ecut=10, kgrid=[1, 1, 1])

Next we set up an initial hydrogen molecule within a box of vacuum.
We use the parameters of the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/)
and DFTK's AtomsBase integration to setup the hydrogen molecule.
We employ a simulation box of 10 bohr times 10 bohr times 10 bohr and a
pseudodojo pseudopotential.

In [2]:
using LinearAlgebra
using Unitful
using UnitfulAtomic

r0 = 1.4   # Initial bond length in Bohr
a  = 10.0  # Box size in Bohr

cell_vectors = [[a, 0, 0]u"bohr", [0, a, 0]u"bohr", [0, 0, a]u"bohr"]
h2_crude = periodic_system([:H => [0, 0, 0.]u"bohr",
                            :H => [r0, 0, 0]u"bohr"],
                           cell_vectors)

FlexibleSystem(H₂, periodicity = TTT):
    cell_vectors      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [       0,        0,        0]u"a₀")
    Atom(H,  [     1.4,        0,        0]u"a₀")


Finally we call `minimize_energy!` to start the geometry optimisation.
We use `verbosity=2` to get some insight into the minimisation.
With `verbosity=1` only a summarising table would be printed and with
`verbosity=0` (default) the minimisation would be quiet.

In [3]:
using GeometryOptimization
results = minimize_energy!(h2_crude, calc; tol_forces=2e-6, verbosity=2)
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -1.110743573144                   -0.82   0.80    8.0    4.13s
  2   -1.117206779913       -2.19       -1.89   0.80    1.0    2.04s
  3   -1.117251863326       -4.35       -2.71   0.80    1.0   56.7ms
  4   -1.117252477060       -6.21       -3.50   0.80    1.0   18.0ms
  5   -1.117252498961       -7.66       -4.02   0.80    1.0   19.4ms
  6   -1.117252500787       -8.74       -5.00   0.80    1.0   26.3ms
  7   -1.117252500844      -10.24       -5.53   0.80    1.0   18.6ms
  8   -1.117252500846      -11.79       -6.45   0.80    1.0   21.1ms
  9   -1.117252500846      -13.06       -6.63   0.80    2.0   21.2ms
 10   -1.117252500846      -14.10       -6.82   0.80    1.0   22.9ms
 11   -1.117252500846      -14.24       -8.13   0.80    1.0    336ms
 12   -1.117252500846      -14.75       -8.30   0.80    2.0   21.4ms
 13   -1.117252500846      -14.57  

Structure after optimisation (note that the atom has wrapped around)

In [4]:
results.system

FlexibleSystem(H₂, periodicity = TTT):
    cell_vectors      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [-0.0431826, -6.8989e-11, 3.10844e-11]u"a₀")
    Atom(H,  [ 1.44318, -6.85458e-11, 3.3708e-11]u"a₀")


Compute final bond length:

In [5]:
rmin = norm(position(results.system[1]) - position(results.system[2]))
println("Optimal bond length: ", rmin)

Optimal bond length: 1.4863652616121568 a₀


Our results (1.486 Bohr) agrees with the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/).